# Honors Project 3
- Anna Kopec
- CSE 3500

## Code

In [ ]:
import numpy as np
import math
import scipy

import data

In [ ]:
from google.colab import files
# upload files from data_and_sols folder
uploaded = files.upload()

Saving casino1.txt to casino1 (1).txt
Saving casino1_sols.txt to casino1_sols (1).txt
Saving casino2.txt to casino2 (1).txt
Saving casino3.txt to casino3 (1).txt
Saving casino3_sols.txt to casino3_sols (1).txt
Saving casino2_sols.txt to casino2_sols (1).txt


### initialize stuff
* reformat data
* 2 hidden states
* 6 possible observations


In [ ]:
def data_format(f, i):
  with open(f, 'r') as file:
    lines = file.readlines()
  data = [num.strip() for num in lines[i]]
  data.pop()
  return data

In [ ]:
inputs = [data_format(f'casino{i}.txt', 0) for i in range(1,4)]
outputs = [data_format(f'casino{i}_sols.txt', 1) for i in range(1,4)]

% accuracy function (correct predictions/total predictions)

In [ ]:
def acc(A, B):
  n = len(B)
  correct = 0
  for i in range(n):
    if A[i] == B[i]:
      correct += 1
  return ((correct)/n)*100


Initialize probabilities

In [ ]:
dice = {0: 'F', 1: 'L'}
dice_rolls = ['1', '2', '3', '4', '5', '6']

# emmision probability

fair_EP = np.full(6, 1/6)
load_EP = np.full(5, 1/10)
load_EP = np.append(load_EP, 1/2)

EP = {0:{dice_rolls[i]: fair_EP[i] for i in range(6)},
      1: {dice_rolls[i]: load_EP[i] for i in range(6)}}

# transition probability

TP = {0:[0.95, 0.05], 1:[0.1, 0.9]}

# equally likely to start w L or F
init = [0.5, 0.5]


In [ ]:
EP

{0: {'1': 0.16666666666666666,
  '2': 0.16666666666666666,
  '3': 0.16666666666666666,
  '4': 0.16666666666666666,
  '5': 0.16666666666666666,
  '6': 0.16666666666666666},
 1: {'1': 0.1, '2': 0.1, '3': 0.1, '4': 0.1, '5': 0.1, '6': 0.5}}

In [ ]:
TP

{0: [0.95, 0.05], 1: [0.1, 0.9]}

### Viterbi Alg

In [ ]:
def Viterbi(S, R, I, TP, EP, T):
    '''Viterbi algorithm
    alg uses logs to avoid underflow '''

    T_len = len(T)
    prob = np.zeros((T_len, S), dtype=np.float64)
    prev = np.zeros((T_len, S), dtype=np.float64)

    # initial timestep

    for s in range(S):
        prob[0][s] = math.log(I[s]) + math.log(EP[s][T[0]])

    # forawrd pass
    for t in range(1, T_len):
        for s in range(S):
            max_prob = -np.inf
            max_state = -1
            for a in range(S):
                n_prob = prob[t-1][a] + math.log(TP[a][s]) + math.log(EP[s][T[t]])
                if n_prob > max_prob:
                    max_prob = n_prob
                    max_state = a
            prob[t][s] = max_prob
            prev[t][s] = max_state

    # initialize hidden state path
    path = np.zeros(T_len, dtype=int)
    path[T_len-1] = np.argmax(prob[T_len-1])

    # use back pointers to traceback sequence of hidden states
    for t in range(T_len-2, -1, -1):
        path[t] = prev[t+1][path[t+1]]

    # return most likely sequence of hidden states
    return [dice[path[i]] for i in range(len(path))]

Test Viterbi

In [ ]:
accs = []
# go thru all 3 cases
for i in range(3):
  gen = Viterbi(2, 6, init, TP, EP, inputs[i])
  accs.append(acc(gen, outputs[i]))
print(f'mean prediction accuracy is: {np.mean(accs)}%')

mean prediction accuracy is: 83.31773602741345%


### Forward-Backward Alg

In [ ]:
def ForwardBackward(S, I, TP, EP, T):
    ''' Forward-Backward Algorithm
    Alg uses log-sum exp trick to avoid under/overflow'''

    T_len = len(T)
    forward = np.zeros((T_len, S), dtype=np.float64)
    backward = np.zeros((T_len, S), dtype=np.float64)

    # initial time step
    for s in range(S):
        forward[0, s] = np.log(I[s]) + (EP[s][int(T[0])-1])
        backward[T_len-1, s] = np.log(1)

    # forward pass
    for t in range(1, T_len):
        for s in range(S):
            log_transitions = forward[t-1, :] + (TP[:][s])
            forward[t, s] = (EP[s][int(T[t]) - 1]) + scipy.special.logsumexp(log_transitions)

    # backward pass
    for t in range(T_len - 2, -1, -1):
        for s in range(S):
            log_transitions = TP[s][:] + [EP[s][int(T[t + 1])-1] for s in range(S)] + backward[t + 1, :]
            backward[t, s] = scipy.special.logsumexp(log_transitions)

    # normalize
    FB = np.zeros((T_len, S))
    for t in range(T_len):
        for s in range(S):
            FB[t, s] = forward[t, s] + backward[t, s]
        FB[t, :] = np.exp(FB[t, :] - scipy.special.logsumexp(FB[t, :]))

    # return array of hidden state probability distributions for each timestep
    return FB

Test Forward-Backward

In [ ]:
probs = []

# convert probabilities to log (and slight reformatting on EP nested dict -> EP 2d list)
log_TP = [np.log(p) for p in TP.values()]
log_EP = []
for s in range(2):
  curr = EP[s]
  log_EP.append([np.log(i) for i in curr.values()])

# go thru all 3 cases
for i in range(3):
  gen = ForwardBackward(2, init, log_TP, log_EP, inputs[i])
  probs.append(gen)



In [ ]:
# convert probability distribution to sequence based on max prob at each t step
sequences = []
for prob in probs:
  sequence = []
  for i in prob:
    if i[0] > i[1]:
      sequence.append('F')
    else:
      sequence.append('L')
  sequences.append(sequence)

accs = []
for i in range(3):
  accs.append(acc(sequences[i], outputs[i]))
print(f'mean prediction accuracy is: {np.mean(accs)}%')



mean prediction accuracy is: 86.98294537004215%


## Written Analysis

*Implement the Viterbi and forward-backward algorithms using the previously defined HMM
to find the most probable explanation of hidden states that generate the series of dice rolls
on HuskyCT. Compare your solutions with the ground truth. In general, discuss when you
would prefer Viterbi over forward-backward and vice-versa. Is it always the best idea to
return the most likely explanation? Why or why not? We included two chapters on Hidden
Markov models on HuskyCT for additional perspectives on HMMs*

After running it on all provided casino data:
* Viterbi algorithm had a mean accuracy of ~83% to the solutions
* FB had a mean accuarcy of ~87% to the solutions

Viterbi is able to find the most probable sequence of hidden states that correspond with a given sequence of occurences. This makes it better for situations where you need to focus on the sequential/temporal aspect of hidden states, such as in DNA. On the other hand, F-B finds hidden state probability distributions for any given position in the sequence, given a sequence of occurences, which makes it better for when you need to focus on individual data points and probabilities, such as for machine learning.

I don't think that it is always ideal to return the most likely explanation (most likely sequence of hidden states). If the difference in hidden state probabilities at a given timestep are low (e.g. say, at timestep t, state A and B have probabilities of 0.49 and 0.51 respectively), then simply taking the max is unreliable. To preserve this nuance, it would be better just to have a series of probabilities.